In [1]:
from fpdf import FPDF
import requests
from PIL import Image
from io import BytesIO
from collections import defaultdict
from fpdf.enums import XPos, YPos

In [37]:
# ----- Variables de ejemplo

nombre_ruta = "Mi Ruta Superguay"
nombre_etapa = "Etapa 23123"
longitud = "32.213 km"
provincia = "Madrid"
CCAA = "Comunidad de Madrid"
imagen_ruta = "mapa_espana.png"

especies = [
    {
        "idtaxon": 14070,
        "nombre_aceptado": "Xema sabini (Sabine, 1819)",
        "nombre_comun": "Gaviotón",
        "taxonomicgroup": "Plantas vasculares",
        "descripcion": "La gaviota de Sabine es una gaviota pequeña con alas largas y vuelo elegante. Es fácil de identificar por su cola ligeramente ahorquillada, algo que no tienen otras gaviotas europeas. En verano, la cabeza del adulto es gris con un collar negro fino que destaca sobre el cuello blanco. En invierno, su cabeza se vuelve más blanca con manchas oscuras. Su cuerpo es blanco por debajo y gris por encima. Lo más llamativo son sus alas: tienen un triángulo blanco en la parte superior que contrasta con el gris del resto del ala y el negro de la punta. Las gaviotas jóvenes tienen un color marrón-grisáceo oscuro en la espalda y la cabeza, pero mantienen el mismo dibujo característico en las alas.",
        "photo_identifier": "https://inaturalist-open-data.s3.amazonaws.com/photos/51996507/original.jpeg",
        "rightsHolder": "Josep Gesti",
        "isinvasive": "No invasora",
        "conservationstatus": "CR (En peligro crítico)"

    },
    {
        "idtaxon": 10479,
        "nombre_aceptado": "Amandava amandava (Linnaeus, 1758)",
        "nombre_comun": "Bengalí rojo",
        "taxonomicgroup": "Aves",
        "descripcion": "Este charrán es un ave de tamaño mediano, de color pálido. Su espalda y alas son grises claras, con la parte baja de la espalda y la cola blancas, esta última con una forma de horquilla. En verano, los adultos tienen la cabeza negra, pico rojo con la punta negra y patas rojas. En invierno, su frente se vuelve blanca y el pico oscuro. Los jóvenes tienen un tono ocre en la espalda, con marcas marrones y blancas, pico oscuro con la parte inferior naranja y patas naranja pálido. Los jóvenes también tienen una línea oscura en el borde del ala. Una característica útil para identificarlo es que, cuando está posado, la cola no sobresale más allá de las alas. Su vuelo es un poco más pesado en comparación con otros charranes parecidos.",
        "photo_identifier": "https://inaturalist-open-data.s3.amazonaws.com/photos/350341158/original.jpeg",
        "rightsHolder": "Robert Martin",
        "isinvasive": "Invasora",
        "conservationstatus": "VU (Vulnerable)"
    },
    {
        "idtaxon": 10708,
        "nombre_aceptado": "Geomalacus anguiformis (Morelet, 1845)",
        "nombre_comun": None,
        "taxonomicgroup": "Invertebrados",
        "descripcion": "Esta babosa mide unos 7 centímetros de largo. Las babosas jóvenes son de color entre negro y azulado, con pequeños bultos blancos y cuatro líneas casi negras en el lomo. Conforme crecen, se aclaran y el azul se vuelve más fuerte, escondiendo los bultos blancos. Las líneas del lomo se hacen más gruesas. Las babosas adultas tienen el lomo marrón, los lados amarillentos y cuatro líneas marrones oscuras, casi negras, en el lomo. La parte inferior del pie es blanquecina, y la baba que produce es amarillenta.",
        "photo_identifier": "https://inaturalist-open-data.s3.amazonaws.com/photos/52000298/original.jpeg",
        "rightsHolder": "Vsevolud Rudyi",
        "isinvasive": "No invasora",
        "conservationstatus": "LC (Preocupación menor)"
    },
    {
        "idtaxon": 10715,
        "nombre_aceptado": "Plecotus auritus (Linnaeus, 1758)",
        "nombre_comun": "Murciélago orejudo dorado",
        "taxonomicgroup": "Mamíferos",
        "descripcion": "Este murciélago es de tamaño mediano y destaca por sus orejas largas y unidas en la base. Dentro de la oreja, tiene una estructura blanquecina y casi transparente. Su pelaje es abundante y largo, de color entre marrón amarillento y marrón grisáceo en la espalda, mientras que en la parte inferior es gris claro con toques amarillentos. Los murciélagos jóvenes tienen un color más pálido y la cara más oscura. Es difícil distinguirlo de otras especies similares a simple vista. Emite sonidos de alta frecuencia mientras busca comida, alcanzando su punto máximo alrededor de 50 kHz.",
        "photo_identifier": "https://inaturalist-open-data.s3.amazonaws.com/photos/286732715/original.jpg",
        "rightsHolder": "Anthony Batista",
        "isinvasive": "Invasora",
        "conservationstatus": "DD (Datos insuficientes)"
    },
    {
        "idtaxon": 10555,
        "nombre_aceptado": "Medusais cuasi (Morelet, 1234)",
        "nombre_comun": "Organismo medusiforme",
        "taxonomicgroup": "Peces",
        "descripcion": "Este organismo marino parece una pequeña medusa gelatinosa, pero sin aguijón. Mide entre 7 y 12 cm de largo y es transparente. Tiene una forma ovalada y aplanada a los lados. Para comer, usa dos grandes labios que tiene a cada lado de la boca, y debajo de ellos hay otros cuatro más pequeños. Lo más llamativo son las ocho bandas a lo largo de su cuerpo formadas por pequeños peines que brillan con colores del arcoíris durante el día y con un color verde brillante por la noche. Tiene cuatro líneas profundas y fáciles de ver en su cuerpo.",
        "photo_identifier": "https://inaturalist-open-data.s3.amazonaws.com/photos/415605212/original.jpg",
        "rightsHolder": "Diego González Dopico",
        "isinvasive": "No invasora",
        "conservationstatus": "NT (Casi amenazado)"
    },
    {
        "idtaxon": 10411,
        "nombre_aceptado": "Algus supremus (Linneaeaus, 213)",
        "nombre_comun": None,
        "taxonomicgroup": "Algas",
        "descripcion": "Es un alga pequeña, de color rojo rosado y muy ramificada, que crece formando grupos densos que parecen algodón, llegando a tener hasta 1 cm de grosor. Se fija al suelo marino o a otras algas mediante pequeñas raíces. Sus tallos son delgados y se levantan, creando una forma como de manto. Al final de muchas de sus ramitas se pueden observar unas células especiales con forma de huevo pequeño. El alga en total mide entre 0.5 y 1.5 cm de largo.",
        "photo_identifier": "https://inaturalist-open-data.s3.amazonaws.com/photos/137401599/original.jpeg",
        "rightsHolder": "Sara Navarro",
        "isinvasive": "No Invasora",
        "conservationstatus": None
    }
] * 3

In [41]:
def recortar_a_cuadrado(imagen):
    ancho, alto = imagen.size
    lado = min(ancho, alto)
    izquierda = (ancho - lado) // 2
    arriba = (alto - lado) // 2
    derecha = izquierda + lado
    abajo = arriba + lado
    return imagen.crop((izquierda, arriba, derecha, abajo))

class PDF(FPDF):
    def footer(self):
        self.set_y(-20)

        # ---- Marca Caminaturaleza ----
        icon_size = 12
        espacio = 2
        text = "Caminaturaleza"
        self.set_font("Helvetica", "", 10)
        text_width = self.get_string_width(text)

        # Calcula el punto x de inicio (pegado al margen derecho)
        x_icon = self.w - self.r_margin - icon_size - espacio - text_width
        y_icon = self.get_y()

        # Imagen sin deformar
        try:
            self.image("logo.png", x=x_icon, y=y_icon, w=icon_size)
        except Exception as e:
            print(f"Error cargando logo: {e}")

        # Centrado vertical preciso del texto respecto al logo
        y_text = y_icon + (icon_size - 2.8) / 3 + 4
        self.set_xy(x_icon + icon_size + espacio, y_text)
        self.cell(text_width, 5, text)

        # ---------- Número de página centrado ----------
        self.set_font("Helvetica", "", 10)
        page_text = f"Página {self.page_no()} de {{nb}}"
        page_width = self.get_string_width(page_text)
        self.set_x((self.w - page_width) / 2)
        self.cell(page_width, 10, page_text)

    def add_info_ruta(self, longitud, provincia, ccaa):
        self.set_font("Helvetica", "", 12)
        self.cell(0, 10, f"Longitud: {longitud}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self.cell(0, 10, f"Provincia: {provincia}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self.cell(0, 10, f"Comunidad Autónoma: {ccaa}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self.ln(10)
    
    def add_portada(self, titulo, subtitulo, imagen_ruta, longitud, provincia, ccaa):
        # ---- Medidas del bloque flotante
        x_ini = 0
        y_ini = 10
        ancho = 170
        alto = 40
        dobladillo = 25  # el pico lateral

        # ---- Sombra debajo
        self.set_fill_color(130, 181, 150)
        self.polygon([
            (x_ini, y_ini),
            (x_ini + ancho + 2, y_ini + 2),
            (x_ini + ancho + dobladillo + 2, y_ini + alto // 2 + 2),
            (x_ini + ancho + 2, y_ini + alto + 2),
            (x_ini, y_ini + alto + 2)
        ], style="F")

        # ---- Banner principal
        self.set_fill_color(160, 211, 180)
        self.polygon([
            (x_ini, y_ini),
            (x_ini + ancho, y_ini),
            (x_ini + ancho + dobladillo, y_ini + alto // 2),
            (x_ini + ancho, y_ini + alto),
            (x_ini, y_ini + alto)
        ], style="F")

        # ---- Texto dentro del banner
        self.set_text_color(0)

        # Cálculo del centro del banner
        center_x = x_ini + (ancho + dobladillo) / 2
        center_y = y_ini + alto / 2

        # Título
        self.set_font("Helvetica", "B", 26)
        titulo_width = self.get_string_width(titulo)
        self.set_xy(center_x - titulo_width / 2, center_y - 10)
        self.cell(titulo_width, 10, titulo)

        # Subtítulo debajo
        self.set_font("Helvetica", "", 18)
        subtitulo_width = self.get_string_width(subtitulo)
        self.set_xy(center_x - subtitulo_width / 2, center_y + 2)
        self.cell(subtitulo_width, 10, subtitulo)

        # ----- Imagen principal
        imagen_y = y_ini + alto + 10
        alto_img = self.h * 0.5
        try:
            self.image(imagen_ruta, x=10, y=imagen_y, w=self.w - 20, h=alto_img)
        except:
            self.set_xy(10, imagen_y)
            self.set_font("Helvetica", "I", 12)
            self.cell(0, 10, "(Imagen no disponible)", ew_x=XPos.LMARGIN, new_y=YPos.NEXT)

        # --- Datos bonitos en dos cajitas iguales, con íconos y texto alineado y wrap
        self.set_y(imagen_y + alto_img + 15)

        box_w = 90
        box_h = 45
        icon_size = 15
        pad = 4       # padding interior de las cajas
        space = 10    # espacio entre cajas

        x_center = self.w / 2
        x1 = x_center - box_w - space / 2
        x2 = x_center + space / 2
        y0 = self.get_y()

        self.set_fill_color(160, 211, 180)

        # ---- Caja 1: Provincia + Comunidad Autónoma
        self.rect(x1, y0, box_w, box_h, style='F')

        line_h = 6
        # --- Provincia
        y_line = y0 + (box_h - line_h) / 4
        self.image("provincia.png", x1 + pad, y_line + (line_h-icon_size)/2, icon_size, icon_size)
        self.set_xy(x1 + pad + icon_size + 2, y_line)
        self.set_font("Helvetica", "B", 12)
        self.cell( self.get_string_width("Provincia: "), line_h, "Provincia: ", new_x=XPos.RIGHT, new_y=YPos.TOP)
        self.set_font("Helvetica", "", 12)
        # wrap dentro de la caja
        self.multi_cell(box_w - (pad + icon_size + 2), line_h, provincia)

        # --- Comunidad Autónoma
        y_line = y0 + (box_h - line_h) / 4 + 2*line_h
        self.image("ccaa.png", x1 + pad, y_line + (line_h-icon_size)/2, icon_size, icon_size)
        self.set_xy(x1 + pad + icon_size + 2, y_line)
        self.set_font("Helvetica", "B", 12)
        self.multi_cell(box_w - (pad + icon_size + 2), line_h, "Comunidad Autónoma:")
        self.set_font("Helvetica", "", 12)
        self.set_x(x1 + pad + icon_size + 2)
        self.multi_cell(box_w - (pad + icon_size + 2), line_h, ccaa)

        # ---- Caja 2: Longitud
        self.rect(x2, y0, box_w, box_h, style='F')

        # La longitud centrada verticalmente
        y_line = y0 + (box_h - line_h) / 2
        self.image("longitud.png", x2 + pad, y_line + (line_h-icon_size)/2, icon_size, icon_size)
        self.set_xy(x2 + pad + icon_size + 2, y_line)
        self.set_font("Helvetica", "B", 12)
        self.cell(self.get_string_width("Longitud: "), line_h, "Longitud: ", new_x=XPos.RIGHT, new_y=YPos.TOP)
        self.set_font("Helvetica", "", 12)
        self.multi_cell(box_w - (pad + icon_size + 2), line_h, longitud)

    def add_grupo_header(self, grupo):
        self.set_font("Helvetica", "B", 18)
        self.set_fill_color(160, 211, 180)
        self.cell(0, 10, f"{grupo}", new_x=XPos.LMARGIN, new_y=YPos.NEXT, fill=True)
        self.ln(5)

    def add_especie(self, idtaxon, nombre_cientifico, nombre_comun, descripcion, url_imagen, autor, isinvasive, conservationstatus):
    # Título con nombre común y nombre científico
        palabras = nombre_cientifico.split()
        nombre_cientifico_cursiva = " ".join(palabras[:2])
        resto = " " + " ".join(palabras[2:]) if len(palabras) > 2 else ""

        self.set_font("Helvetica", "B", 13)

        if nombre_comun:
            self.write(8, f"{nombre_comun} (")
            # Parte en cursiva
            self.set_font("Helvetica", "BI", 13)
            self.write(8, " ".join(nombre_cientifico.split()[:2]))
            # Parte restante del nombre científico (si hay más)
            resto = " " + " ".join(nombre_cientifico.split()[2:]) if len(nombre_cientifico.split()) > 2 else ""
            self.set_font("Helvetica", "B", 13)
            self.write(8, f"{resto})")
        else:
            self.set_font("Helvetica", "BI", 13)
            self.write(8, " ".join(nombre_cientifico.split()[:2]))
            resto = " " + " ".join(nombre_cientifico.split()[2:]) if len(nombre_cientifico.split()) > 2 else ""
            self.set_font("Helvetica", "B", 13)
            self.write(8, resto)

        self.ln(10)

        # Si es invasora, mostrar advertencia justo debajo del nombre
        if isinvasive.strip().lower() == "invasora":
            # Parámetros del triángulo
            tri_size = 5  # tamaño del triángulo
            space_below = 2

            # Posición del triángulo (entre nombre y cuerpo)
            x0 = self.get_x()
            y0 = self.get_y()

            # Coordenadas del triángulo equilátero (punta arriba)
            triangle = [
                (x0, y0 + tri_size),
                (x0 + tri_size, y0 + tri_size),
                (x0 + tri_size / 2, y0)
            ]

            # Dibujar triángulo amarillo
            self.set_fill_color(255, 204, 0)
            self.polygon(triangle, style='F')

            # Dibujar "!" encima del triángulo
            self.set_font("Helvetica", "B", 10)
            self.set_xy(x0 + 0.85, y0 + 2)
            self.cell(tri_size, 3, "!", new_x=XPos.RIGHT, new_y=YPos.TOP)

            # Texto al lado
            self.set_font("Helvetica", "B", 9)
            self.set_xy(x0 + tri_size + 1, y0)
            self.cell(0, 5, "Especie invasora", new_x=XPos.LMARGIN, new_y=YPos.NEXT)

            # Restaurar color y fuente
            self.set_text_color(0)
            self.set_font("Helvetica", "", 11)

            self.ln(space_below)  # Separación antes del cuerpo

        try:
            response = requests.get(url_imagen)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content))
                img = recortar_a_cuadrado(img)

                img_size = 50  # tamaño de la imagen en mm
                x_img = self.get_x()
                y_img = self.get_y()

                # Dibuja imagen a la izquierda
                self.image(img, x=x_img, y=y_img, w=img_size, h=img_size)

                # Mueve el cursor al lado derecho de la imagen
                self.set_xy(x_img + img_size + 5, y_img)
                self.set_font("Helvetica", "", 11)
                self.multi_cell(0, 5, descripcion)

                # Volver a abajo del bloque
                self.set_y(max(self.get_y(), y_img + img_size))

                # Créditos justo debajo de la imagen
                self.set_xy(x_img, y_img + img_size + 1)  # 1 mm de separación
                self.set_font("Helvetica", "I", 9)
                self.cell(0, 6, f"Créditos: {autor}", new_x=XPos.LMARGIN, new_y=YPos.NEXT)

                # Luego, ajustamos el cursor para continuar después de la zona de créditos
                self.set_y(y_img + img_size + 5)  # 8 mm desde la base de la imagen para dejar espacio
        except Exception as e:
            print(f"Error cargando imagen: {e}")

        self.ln(10)

In [42]:
# Agrupar por grupo taxonómico
especies_por_grupo = defaultdict(list)
for especie in especies:
    grupo = especie["taxonomicgroup"]
    especies_por_grupo[grupo].append(especie)

# Crear PDF
pdf = PDF()
pdf.titulo = nombre_ruta
pdf.subtitulo = nombre_etapa

# Establecer márgenes superior e inferior
pdf.set_top_margin(20)                          # Margen superior
pdf.set_auto_page_break(auto=True, margin=20)   # Margen inferior
pdf.set_left_margin(15)     # Margen izquierdo
pdf.set_right_margin(15)    # Margen derecho

# Página inicial
pdf.add_page()
pdf.add_portada(nombre_ruta, nombre_etapa, imagen_ruta, longitud, provincia, CCAA)

# Para cada grupo taxonómico
# Controlar qué grupos ya se han mostrado

grupos_mostrados = set()
especies_por_pagina = 3
contador_en_pagina = 0

pdf.add_page()

# Aplanar lista de especies y mantener orden por grupo
especies_ordenadas = []
for grupo, lista in sorted(especies_por_grupo.items()):
    for especie in lista:
        especies_ordenadas.append((grupo, especie))

for grupo, especie in especies_ordenadas:
    # Añadir grupo taxonómico si no se ha mostrado antes
    if grupo not in grupos_mostrados:
        pdf.add_grupo_header(grupo)
        grupos_mostrados.add(grupo)

    # Añadir especie
    pdf.add_especie(
        idtaxon=especie["idtaxon"],
        nombre_cientifico=especie["nombre_aceptado"],
        nombre_comun=especie["nombre_comun"],
        descripcion=especie["descripcion"],
        url_imagen=especie["photo_identifier"],
        autor=especie["rightsHolder"],
        isinvasive=especie["isinvasive"],
        conservationstatus=especie["conservationstatus"]
    )

    # Cuando se agregue una especie, revisamos si hemos agregado 3
    contador_en_pagina += 1

    # Pasar a la siguiente página si se ha alcanzado 3 especies
    if contador_en_pagina == 3:
        # Solo agregar nueva página si no es la última especie
        if (grupo, especie) != especies_ordenadas[-1]:
            pdf.add_page()
        contador_en_pagina = 0

In [43]:
pdf.output("pdf_salida.pdf")